# Analysis Before implementing CASE1, CASE2 and CASE3

In [23]:
# Calling the Necessary Libararies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, PowerTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
import matplotlib.pyplot as plt
from tabulate import tabulate
import seaborn as sns
from scipy import stats
from sklearn.ensemble import IsolationForest

warnings.filterwarnings('ignore')
print("Ignoring all the warnings -- True")

Ignoring all the warnings -- True


In [24]:
df = pd.read_csv('ChatsAfter.csv')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Psydo_Names     9999 non-null   object
 1   Gender          9999 non-null   object
 2   Query_Asked     9999 non-null   object
 3   Query_Answered  9999 non-null   object
 4   Feedback        9999 non-null   object
dtypes: object(5)
memory usage: 390.7+ KB


In [26]:
df.head()

,Psydo_Names,Gender,Query_Asked,Query_Answered,Feedback
0,Alex Bennett,Female,D,C,Great job
1,Casey Morgan,Female,E,D,Not up to the mark
2,Jordan Quinn,Male,B,C,Well done
3,Taylor Reese,Female,D,A,Not up to the mark
4,Avery Blake,Female,B,C,Very bad performance


In [27]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [28]:
analyzer = SentimentIntensityAnalyzer()
def analyze_sentiment(feedback):
    scores = analyzer.polarity_scores(feedback)
    return 'Positive' if scores['compound'] > 0 else 'Negative'

df['Sentiment_Analysis'] = df['Feedback'].apply(analyze_sentiment)

In [29]:
df

,Psydo_Names,Gender,Query_Asked,Query_Answered,Feedback,Sentiment_Analysis
0,Alex Bennett,Female,D,C,Great job,Positive
1,Casey Morgan,Female,E,D,Not up to the mark,Negative
2,Jordan Quinn,Male,B,C,Well done,Positive
3,Taylor Reese,Female,D,A,Not up to the mark,Negative
4,Avery Blake,Female,B,C,Very bad performance,Negative
...,...,...,...,...,...,...
9994,Parker Casey,Female,A,A,Unacceptable result,Negative
9995,Hunter Jamie,Female,D,E,Excellent performance,Positive
9996,Ellis Cameron,Female,C,D,Poor quality work,Negative
9997,Lee Devin,Male,C,D,Poor quality work,Negative


In [30]:
df['Churn_Rate'] = df['Sentiment_Analysis'].apply(lambda x: 0 if x == 'Negative' else 1)
churn_rate_percentage = (df['Churn_Rate'].sum() / len(df)) * 100
df.head(20)

,Psydo_Names,Gender,Query_Asked,Query_Answered,Feedback,Sentiment_Analysis,Churn_Rate
0,Alex Bennett,Female,D,C,Great job,Positive,1
1,Casey Morgan,Female,E,D,Not up to the mark,Negative,0
2,Jordan Quinn,Male,B,C,Well done,Positive,1
3,Taylor Reese,Female,D,A,Not up to the mark,Negative,0
4,Avery Blake,Female,B,C,Very bad performance,Negative,0
5,Riley Parker,Female,B,E,Needs significant improvement,Positive,1
6,Morgan Lee,Male,D,D,Great job,Positive,1
7,Jamie Ellis,Female,A,B,Excellent performance,Positive,1
8,Cameron Hunter,Male,B,E,Completely wrong,Negative,0
9,Devin Riley,Female,A,E,Very bad performance,Negative,0


In [31]:
import random, string
used_pseudonyms = set()

def generate_unique_pseudonym():
    while True:
        # Generate a random 5-digit number
        number = ''.join(random.choices(string.digits, k=5))
        # Create the pseudonym with a prefix 'TCK' and the random number
        pseudonym = f'TCK{number}'
        if pseudonym not in used_pseudonyms:
            used_pseudonyms.add(pseudonym)
            return pseudonym

# Apply the pseudonym generation only to rows where 'Gender' is not null
df.loc[df['Gender'].notna(), 'Psydo_Names'] = [generate_unique_pseudonym() for _ in range(len(df[df['Gender'].notna()]))]

In [32]:
df.head()

,Psydo_Names,Gender,Query_Asked,Query_Answered,Feedback,Sentiment_Analysis,Churn_Rate
0,TCK12018,Female,D,C,Great job,Positive,1
1,TCK80761,Female,E,D,Not up to the mark,Negative,0
2,TCK01219,Male,B,C,Well done,Positive,1
3,TCK72581,Female,D,A,Not up to the mark,Negative,0
4,TCK71621,Female,B,C,Very bad performance,Negative,0


In [33]:
print(f"Churn Rate: {churn_rate_percentage:.2f}%")

Churn Rate: 75.15%
